# Test KNeighborsRegressor

# Feature Engineering

In [2]:
from sklearn.model_selection import KFold, GridSearchCV, train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.preprocessing import StandardScaler
import time
%matplotlib inline

import xgboost
from sklearn.neighbors import KNeighborsRegressor
from bayes_opt import BayesianOptimization

import warnings
warnings.filterwarnings("ignore")

In [3]:
"""
Contributions from:
DSEverything - Mean Mix - Math, Geo, Harmonic (LB 0.493) 
https://www.kaggle.com/dongxu027/mean-mix-math-geo-harmonic-lb-0-493
JdPaletto - Surprised Yet? - Part2 - (LB: 0.503)
https://www.kaggle.com/jdpaletto/surprised-yet-part2-lb-0-503
hklee - weighted mean comparisons, LB 0.497, 1ST
https://www.kaggle.com/zeemeen/weighted-mean-comparisons-lb-0-497-1st

Also all comments for changes, encouragement, and forked scripts rock

Keep the Surprise Going
"""

import glob, re
import numpy as np
import pandas as pd
from sklearn import *
from datetime import datetime

data = {
    'tra': pd.read_csv('../../../mltestdata/05_recruit/air_visit_data.csv'),
    'as': pd.read_csv('../../../mltestdata/05_recruit/air_store_info.csv'),
    'hs': pd.read_csv('../../../mltestdata/05_recruit/hpg_store_info.csv'),
    'ar': pd.read_csv('../../../mltestdata/05_recruit/air_reserve.csv'),
    'hr': pd.read_csv('../../../mltestdata/05_recruit/hpg_reserve.csv'),
    'id': pd.read_csv('../../../mltestdata/05_recruit/store_id_relation.csv'),
    'tes': pd.read_csv('../../../mltestdata/05_recruit/sample_submission.csv'),
    'hol': pd.read_csv('../../../mltestdata/05_recruit/date_info.csv').rename(columns={'calendar_date':'visit_date'})
    }

data['hr'] = pd.merge(data['hr'], data['id'], how='inner', on=['hpg_store_id'])

for df in ['ar','hr']:
    data[df]['visit_datetime'] = pd.to_datetime(data[df]['visit_datetime'])
    data[df]['visit_datetime'] = data[df]['visit_datetime'].dt.date
    data[df]['reserve_datetime'] = pd.to_datetime(data[df]['reserve_datetime'])
    data[df]['reserve_datetime'] = data[df]['reserve_datetime'].dt.date
    data[df]['reserve_datetime_diff'] = data[df].apply(lambda r: (r['visit_datetime'] - r['reserve_datetime']).days, axis=1)
    tmp1 = data[df].groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].sum().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs1', 'reserve_visitors':'rv1'})
    tmp2 = data[df].groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].mean().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs2', 'reserve_visitors':'rv2'})
    data[df] = pd.merge(tmp1, tmp2, how='inner', on=['air_store_id','visit_date'])

data['tra']['visit_date'] = pd.to_datetime(data['tra']['visit_date'])
data['tra']['dow'] = data['tra']['visit_date'].dt.dayofweek
data['tra']['year'] = data['tra']['visit_date'].dt.year
data['tra']['month'] = data['tra']['visit_date'].dt.month
data['tra']['visit_date'] = data['tra']['visit_date'].dt.date

data['tes']['visit_date'] = data['tes']['id'].map(lambda x: str(x).split('_')[2])
data['tes']['air_store_id'] = data['tes']['id'].map(lambda x: '_'.join(x.split('_')[:2]))
data['tes']['visit_date'] = pd.to_datetime(data['tes']['visit_date'])
data['tes']['dow'] = data['tes']['visit_date'].dt.dayofweek
data['tes']['year'] = data['tes']['visit_date'].dt.year
data['tes']['month'] = data['tes']['visit_date'].dt.month
data['tes']['visit_date'] = data['tes']['visit_date'].dt.date

unique_stores = data['tes']['air_store_id'].unique()
stores = pd.concat([pd.DataFrame({'air_store_id': unique_stores, 'dow': [i]*len(unique_stores)}) for i in range(7)], axis=0, ignore_index=True).reset_index(drop=True)

#sure it can be compressed...
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].min().rename(columns={'visitors':'min_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].mean().rename(columns={'visitors':'mean_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].median().rename(columns={'visitors':'median_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].max().rename(columns={'visitors':'max_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].count().rename(columns={'visitors':'count_observations'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 

stores = pd.merge(stores, data['as'], how='left', on=['air_store_id']) 
# NEW FEATURES FROM Georgii Vyshnia
stores['air_genre_name'] = stores['air_genre_name'].map(lambda x: str(str(x).replace('/',' ')))
stores['air_area_name'] = stores['air_area_name'].map(lambda x: str(str(x).replace('-',' ')))
lbl = preprocessing.LabelEncoder()
for i in range(10):
    stores['air_genre_name'+str(i)] = lbl.fit_transform(stores['air_genre_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ''))
    stores['air_area_name'+str(i)] = lbl.fit_transform(stores['air_area_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ''))
stores['air_genre_name'] = lbl.fit_transform(stores['air_genre_name'])
stores['air_area_name'] = lbl.fit_transform(stores['air_area_name'])

data['hol']['visit_date'] = pd.to_datetime(data['hol']['visit_date'])
data['hol']['day_of_week'] = lbl.fit_transform(data['hol']['day_of_week'])
data['hol']['visit_date'] = data['hol']['visit_date'].dt.date
train = pd.merge(data['tra'], data['hol'], how='left', on=['visit_date']) 
test = pd.merge(data['tes'], data['hol'], how='left', on=['visit_date']) 

train = pd.merge(train, stores, how='left', on=['air_store_id','dow']) 
test = pd.merge(test, stores, how='left', on=['air_store_id','dow'])

for df in ['ar','hr']:
    train = pd.merge(train, data[df], how='left', on=['air_store_id','visit_date']) 
    test = pd.merge(test, data[df], how='left', on=['air_store_id','visit_date'])

train['id'] = train.apply(lambda r: '_'.join([str(r['air_store_id']), str(r['visit_date'])]), axis=1)

train['total_reserv_sum'] = train['rv1_x'] + train['rv1_y']
train['total_reserv_mean'] = (train['rv2_x'] + train['rv2_y']) / 2
train['total_reserv_dt_diff_mean'] = (train['rs2_x'] + train['rs2_y']) / 2

test['total_reserv_sum'] = test['rv1_x'] + test['rv1_y']
test['total_reserv_mean'] = (test['rv2_x'] + test['rv2_y']) / 2
test['total_reserv_dt_diff_mean'] = (test['rs2_x'] + test['rs2_y']) / 2

# NEW FEATURES FROM JMBULL
train['date_int'] = train['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
test['date_int'] = test['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
train['var_max_lat'] = train['latitude'].max() - train['latitude']
train['var_max_long'] = train['longitude'].max() - train['longitude']
test['var_max_lat'] = test['latitude'].max() - test['latitude']
test['var_max_long'] = test['longitude'].max() - test['longitude']

# NEW FEATURES FROM Georgii Vyshnia
train['lon_plus_lat'] = train['longitude'] + train['latitude'] 
test['lon_plus_lat'] = test['longitude'] + test['latitude']

lbl = preprocessing.LabelEncoder()
train['air_store_id2'] = lbl.fit_transform(train['air_store_id'])
test['air_store_id2'] = lbl.transform(test['air_store_id'])

col = [c for c in train if c not in ['id', 'air_store_id', 'visit_date','visitors']]
train = train.fillna(-1)
test = test.fillna(-1)

In [4]:
for c, dtype in zip(train.columns, train.dtypes):	
    if dtype == np.float64:		
        train[c] = train[c].astype(np.float32)

for c, dtype in zip(test.columns, test.dtypes):	
    if dtype == np.float64:		
        test[c] = test[c].astype(np.float32)

Set Golden week flag and Post Golden week flag.

In [5]:
combine = [train, test]
gw_list = ['2016-04-29','2016-04-30','2016-05-01','2016-05-02','2016-05-03','2016-05-04','2016-05-05','2017-04-29','2017-04-30','2017-05-01','2017-05-02','2017-05-03','2017-05-04','2017-05-05']
post_gw_list=['2016-05-06']
train['gw_flg'] = 0
train['post_gw_flg'] = 0
test['gw_flg'] = 0
test['post_gw_flg'] = 0
update_gw_list = [["0" for i in range(3)] for j in range(len(gw_list))]
update_post_gw_list = [["0" for i in range(3)] for j in range(len(post_gw_list))]

In [6]:
from datetime import date
for index, gw_date in enumerate(gw_list):
    temp_list = gw_date.split("-")
    for col_i, temp_figure in enumerate(temp_list):
        update_gw_list[index][col_i]=int(temp_figure)
        
    #print("{}  {}  {}".format(update_list[index][0],update_list[index][1],update_list[index][2]))
    
for index, gw_date in enumerate(post_gw_list):
    temp_list = gw_date.split("-")
    for col_i, temp_figure in enumerate(temp_list):
        update_post_gw_list[index][col_i]=int(temp_figure)

for dataset in combine:
    for index in range(len(update_gw_list)):
        dataset.loc[dataset.visit_date == date(update_gw_list[index][0],update_gw_list[index][1],update_gw_list[index][2]), 'gw_flg'] = 1
        
for dataset in combine:
    for index in range(len(update_post_gw_list)):
        dataset.loc[dataset.visit_date == date(update_post_gw_list[index][0],update_post_gw_list[index][1],update_post_gw_list[index][2]), 'post_gw_flg'] = 1     

In [7]:
drop_cols=['visitors','air_store_id','visit_date','id']
y_train=train['visitors']
x_train=train.drop(drop_cols, axis=1)

x_test=test.copy()
x_test=x_test.drop(drop_cols, axis=1)

In [8]:
y = train.visitors
train_input = train.copy()
test_input = test.copy()

drop_cols=['visitors','air_store_id','visit_date','id']
train_input=train_input.drop(drop_cols, axis=1)
test_input=test_input.drop(drop_cols, axis=1)

# Modeling

## KNeighborsRegressor

In [9]:
#Define a evaluation function

def rmsle(preds, true):
    rmsle = np.sqrt(mean_squared_error(np.log1p(true), np.log1p(preds)))
    return float(rmsle)

In [10]:
#Define a evaluation matrix 
from sklearn.metrics.scorer import make_scorer
RMSLE = make_scorer(rmsle)

In [11]:
# Define a function for comparing predictions and true data.
def compare_result(preds, true):
    compare = pd.DataFrame({"test_id": true.index,
                           "real_cost": true,
                           "pred_cost": preds})
    compare = compare[["test_id", "real_cost", "pred_cost"]].reset_index(drop=True)
    
    compare["error_percent_(%)"] = np.abs(compare.real_cost - compare.pred_cost) / compare.real_cost * 100
    
    return compare

# Cross validation with KNeighborsRegressor

In [12]:
def cross_validate_sklearn(clf, x_train, y_train , x_test, kf,scale=False, verbose=True, scoreonly=False):
    start_time=time.time()
    
    # initialise the size of out-of-fold train an test prediction
    train_pred = np.zeros((x_train.shape[0]))
    test_pred = np.zeros((x_test.shape[0]))

    # self-defined eval metric
    # f(preds: array, train_data: Dataset) -> name: string, value: array, is_higher_better: bool
    # binary error
    def feval_rmsle(preds, train_data):
        preds = np.expm1(preds)
        true = np.expm1(train_data.get_label())
        #return 'rmsle', rmsle(true, preds), False

        return 'rmsle', rmsle(preds, true), False
    
    # use the kfold object to generate the required folds
    for i, (train_index, val_index) in enumerate(kf.split(x_train, y_train)):
        # generate training folds and validation fold
        x_train_kf, x_val_kf = x_train.loc[train_index, :], x_train.loc[val_index, :]

        #y_train_kf, y_val_kf = y_train[train_index], y_train[test_index]
        y_train_kf, y_val_kf = np.log1p(y_train[train_index]), np.log1p(y_train[val_index])
        x_test_kf=x_test.copy()

        # perform scaling if required i.e. for linear algorithms
        if scale:
            scaler = StandardScaler().fit(x_train_kf.values)
            x_train_kf_values = scaler.transform(x_train_kf.values)
            x_val_kf_values = scaler.transform(x_val_kf.values)
            x_test_values = scaler.transform(x_test.values)
        else:
            x_train_kf_values = x_train_kf.values
            x_val_kf_values = x_val_kf.values
            x_test_values = x_test.values
        
        # fit the input classifier and perform prediction.
        clf.fit(x_train_kf_values, y_train_kf.values)

        #val_pred=clf.predict_proba(x_val_kf_values)[:,1]
        val_pred=np.expm1(clf.predict(x_val_kf_values))
        #train_pred[test_index] += val_pred
        train_pred[val_index] += val_pred

        #y_test_preds = clf.predict_proba(x_test_values)[:,1]
        y_test_preds = np.expm1(clf.predict(x_test_values))
        test_pred += y_test_preds

        #fold_auc = roc_auc_score(y_val_kf.values, val_pred)
        #fold_gini_norm = auc_to_gini_norm(fold_auc)
        fold_rmsle = rmsle(train_pred[val_index],np.expm1(y_val_kf.values))

        if verbose:
            print('fold cv {} RMSLE score is {:.6f}'.format(i, fold_rmsle))

    test_pred /= kf.n_splits

    #cv_auc = roc_auc_score(y_train, train_pred)
    #cv_gini_norm = auc_to_gini_norm(cv_auc)
    cv_score = rmsle(train_pred, y_train)
    
    if verbose:
        print('cv RMSLE score is {:.6f}'.format(cv_score))
        end_time = time.time()
        print("it takes %.3f seconds to perform cross validation" % (end_time - start_time))

    if scoreonly:
        return cv_score
    else:
        return cv_score, train_pred, test_pred


In [13]:
kf=StratifiedKFold(n_splits=5, shuffle=True, random_state=2018)

In [14]:
# cluster 3 with scaling
knr = KNeighborsRegressor(n_jobs=-1, n_neighbors=3)

outcomes =cross_validate_sklearn(knr, x_train, y_train ,x_test, kf, scale=True, verbose=True)

knr_cv=outcomes[0]
knr_train_pred=outcomes[1]
knr_test_pred=outcomes[2]

knr_train_pred_df=pd.DataFrame(columns=['visitors'], data=knr_train_pred)
knr_test_pred_df=pd.DataFrame(columns=['visitors'], data=knr_test_pred)

fold cv 0 RMSLE score is 0.556426
fold cv 1 RMSLE score is 0.555690
fold cv 2 RMSLE score is 0.549253
fold cv 3 RMSLE score is 0.552633
fold cv 4 RMSLE score is 0.546384
cv RMSLE score is 0.552094
it takes 2933.039 seconds to perform cross validation


In [27]:
# Cluster 4 *no scale
knr = KNeighborsRegressor(n_jobs=-1, n_neighbors=4)

outcomes =cross_validate_sklearn(knr, x_train, y_train ,x_test, kf, scale=False, verbose=True)

knr_cv=outcomes[0]
knr_train_pred=outcomes[1]
knr_test_pred=outcomes[2]

knr_train_pred_df=pd.DataFrame(columns=['visitors'], data=knr_train_pred)
knr_test_pred_df=pd.DataFrame(columns=['visitors'], data=knr_test_pred)

fold cv 0 RMSLE score is 0.557567
fold cv 1 RMSLE score is 0.552856
fold cv 2 RMSLE score is 0.551196
fold cv 3 RMSLE score is 0.549202
fold cv 4 RMSLE score is 0.547743
cv RMSLE score is 0.551727
it takes 1447.553 seconds to perform cross validation


In [31]:
# Cluster 4 with scaling
knr = KNeighborsRegressor(n_jobs=-1, n_neighbors=4)

outcomes =cross_validate_sklearn(knr, x_train, y_train ,x_test, kf, scale=True, verbose=True)

knr_cv=outcomes[0]
knr_train_pred=outcomes[1]
knr_test_pred=outcomes[2]

knr_train_pred_df=pd.DataFrame(columns=['visitors'], data=knr_train_pred)
knr_test_pred_df=pd.DataFrame(columns=['visitors'], data=knr_test_pred)

fold cv 0 RMSLE score is 0.541592
fold cv 1 RMSLE score is 0.539625
fold cv 2 RMSLE score is 0.533373
fold cv 3 RMSLE score is 0.537068
fold cv 4 RMSLE score is 0.532714
cv RMSLE score is 0.536889
it takes 3476.677 seconds to perform cross validation


In [15]:
# Cluster 5 with scaling
knr = KNeighborsRegressor(n_jobs=-1, n_neighbors=5)

outcomes =cross_validate_sklearn(knr, x_train, y_train ,x_test, kf, scale=True, verbose=True)

knr_cv=outcomes[0]
knr_train_pred=outcomes[1]
knr_test_pred=outcomes[2]

knr_train_pred_df=pd.DataFrame(columns=['visitors'], data=knr_train_pred)
knr_test_pred_df=pd.DataFrame(columns=['visitors'], data=knr_test_pred)

fold cv 0 RMSLE score is 0.532660
fold cv 1 RMSLE score is 0.530332
fold cv 2 RMSLE score is 0.524591
fold cv 3 RMSLE score is 0.528426
fold cv 4 RMSLE score is 0.524071
cv RMSLE score is 0.528030
it takes 2813.101 seconds to perform cross validation


In [16]:
# Cluster 6 with scaling
knr = KNeighborsRegressor(n_jobs=-1, n_neighbors=6)

outcomes =cross_validate_sklearn(knr, x_train, y_train ,x_test, kf, scale=True, verbose=True)

knr_cv=outcomes[0]
knr_train_pred=outcomes[1]
knr_test_pred=outcomes[2]

knr_train_pred_df=pd.DataFrame(columns=['visitors'], data=knr_train_pred)
knr_test_pred_df=pd.DataFrame(columns=['visitors'], data=knr_test_pred)

fold cv 0 RMSLE score is 0.526898
fold cv 1 RMSLE score is 0.524095
fold cv 2 RMSLE score is 0.518715
fold cv 3 RMSLE score is 0.522726
fold cv 4 RMSLE score is 0.518273
cv RMSLE score is 0.522155
it takes 15778.621 seconds to perform cross validation


In [17]:
# Cluster 7 with scaling
knr = KNeighborsRegressor(n_jobs=-1, n_neighbors=7)

outcomes =cross_validate_sklearn(knr, x_train, y_train ,x_test, kf, scale=True, verbose=True)

knr_cv=outcomes[0]
knr_train_pred=outcomes[1]
knr_test_pred=outcomes[2]

knr_train_pred_df=pd.DataFrame(columns=['visitors'], data=knr_train_pred)
knr_test_pred_df=pd.DataFrame(columns=['visitors'], data=knr_test_pred)

fold cv 0 RMSLE score is 0.523658
fold cv 1 RMSLE score is 0.521276
fold cv 2 RMSLE score is 0.515578
fold cv 3 RMSLE score is 0.519137
fold cv 4 RMSLE score is 0.514863
cv RMSLE score is 0.518917
it takes 3800.235 seconds to perform cross validation


# GridSearch

In [13]:
from sklearn.metrics import mean_squared_error, r2_score, make_scorer

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

scoring_fnc=make_scorer(rmse)

In [14]:
knr = KNeighborsRegressor(n_jobs=-1, n_neighbors=4)

In [ ]:
#Hyper Parameters Set
params = {'n_neighbors':[5,6,7],
          'leaf_size':[2,3,5],
          'weights':['uniform', 'distance'],
          'algorithm':['auto', 'ball_tree','kd_tree','brute'],
          'n_jobs':[-1]}
#Making models with hyper parameters sets

model1 = GridSearchCV(knr, param_grid=params, n_jobs=1,scoring=scoring_fnc,cv=5,verbose=3)
#Learning
model1.fit(train_input,np.log1p(y))
#The best hyper parameters set
print("Best Hyper Parameters:\n",model1.best_params_)
#Prediction
prediction=model1.predict(test_input)
#importing the metrics module

from sklearn import metrics
#evaluation(Accuracy)
#print("Accuracy:",metrics.accuracy_score(prediction,test_y))
#evaluation(Confusion Metrix)
#print("Confusion Metrix:\n",metrics.confusion_matrix(prediction,test_y))

Fitting 5 folds for each of 72 candidates, totalling 360 fits
[CV] algorithm=auto, leaf_size=2, n_jobs=-1, n_neighbors=5, weights=uniform 
[CV]  algorithm=auto, leaf_size=2, n_jobs=-1, n_neighbors=5, weights=uniform, score=0.666111905261168, total=  12.0s
[CV] algorithm=auto, leaf_size=2, n_jobs=-1, n_neighbors=5, weights=uniform 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   24.9s remaining:    0.0s


[CV]  algorithm=auto, leaf_size=2, n_jobs=-1, n_neighbors=5, weights=uniform, score=0.6925398526999745, total= 5.3min
[CV] algorithm=auto, leaf_size=2, n_jobs=-1, n_neighbors=5, weights=uniform 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  6.3min remaining:    0.0s


[CV]  algorithm=auto, leaf_size=2, n_jobs=-1, n_neighbors=5, weights=uniform, score=0.6800161775155649, total= 4.9min
[CV] algorithm=auto, leaf_size=2, n_jobs=-1, n_neighbors=5, weights=uniform 
[CV]  algorithm=auto, leaf_size=2, n_jobs=-1, n_neighbors=5, weights=uniform, score=0.6701719564017073, total= 4.8min
[CV] algorithm=auto, leaf_size=2, n_jobs=-1, n_neighbors=5, weights=uniform 
[CV]  algorithm=auto, leaf_size=2, n_jobs=-1, n_neighbors=5, weights=uniform, score=0.65008721096032, total= 4.8min
[CV] algorithm=auto, leaf_size=2, n_jobs=-1, n_neighbors=5, weights=distance 
[CV]  algorithm=auto, leaf_size=2, n_jobs=-1, n_neighbors=5, weights=distance, score=0.6662332412701135, total=  12.1s
[CV] algorithm=auto, leaf_size=2, n_jobs=-1, n_neighbors=5, weights=distance 
[CV]  algorithm=auto, leaf_size=2, n_jobs=-1, n_neighbors=5, weights=distance, score=0.6930461823960394, total= 4.8min
[CV] algorithm=auto, leaf_size=2, n_jobs=-1, n_neighbors=5, weights=distance 
[CV]  algorithm=auto, 

# Velification

#### Test #1

In [14]:
xgb_params = {
    "objective" : "reg:linear",
    "eval_metric" : "rmse",
    "nthread": 4,
    "seed" : 0,
    'silent': 1,
    "colsample_bytree"  :  0.5555, 
    "gamma": 1.8385,
    "learning_rate": 0.2968,
    "max_delta_step": 8.3539,
    "max_depth": 10,#9.6366,
    "max_features": 0.6203,
    "min_child_weight": 8.1246,
    "min_samples_split": 16.2850,
    "n_estimators": 14.1175,
    "subsample": 0.7658,
    "seed": 0,
    'silent': 1
}

outcomes=cross_validate_xgb(xgb_params, train_input, y, test_input, kf, verbose_eval=False)

xgb_cv=outcomes[0]
xgb_train_pred=outcomes[1]
xgb_test_pred=outcomes[2]

xgb_train_pred_df=pd.DataFrame(columns=['visitors'], data=xgb_train_pred)
xgb_test_pred_df=pd.DataFrame(columns=['visitors'], data=xgb_test_pred)
print("Done")

fold cv 0 rmsle score is 0.497094
fold cv 1 rmsle score is 0.501248
fold cv 2 rmsle score is 0.496586
cv rmsle score is 0.498314
it takes 307.088 seconds to perform cross validation
Done


# Saving data

In [18]:
lv1_knr_train_pred = knr_train_pred_df.copy()
lv1_knr_train_pred.to_csv('lv1_knr_train_pred.csv', index=False)

lv1_knr_test_pred = knr_test_pred_df.copy()
lv1_knr_test_pred.to_csv('lv1_knr_test_pred.csv', index=False)

# Last work with weight

In [37]:
sub1 = sub.copy()

In [38]:
# from hklee
# https://www.kaggle.com/zeemeen/weighted-mean-comparisons-lb-0-497-1st/code
dfs = { re.search('/([^/\.]*)\.csv', fn).group(1):
    pd.read_csv(fn)for fn in glob.glob('../../../mltestdata/05_recruit/*.csv')}

for k, v in dfs.items(): locals()[k] = v

wkend_holidays = date_info.apply(
    (lambda x:(x.day_of_week=='Sunday' or x.day_of_week=='Saturday') and x.holiday_flg==1), axis=1)
date_info.loc[wkend_holidays, 'holiday_flg'] = 0
date_info['weight'] = ((date_info.index + 1) / len(date_info)) ** 5  

visit_data = air_visit_data.merge(date_info, left_on='visit_date', right_on='calendar_date', how='left')
visit_data.drop('calendar_date', axis=1, inplace=True)
visit_data['visitors'] = visit_data.visitors.map(pd.np.log1p)

wmean = lambda x:( (x.weight * x.visitors).sum() / x.weight.sum() )
visitors = visit_data.groupby(['air_store_id', 'day_of_week', 'holiday_flg']).apply(wmean).reset_index()
visitors.rename(columns={0:'visitors'}, inplace=True) # cumbersome, should be better ways.

sample_submission['air_store_id'] = sample_submission.id.map(lambda x: '_'.join(x.split('_')[:-1]))
sample_submission['calendar_date'] = sample_submission.id.map(lambda x: x.split('_')[2])
sample_submission.drop('visitors', axis=1, inplace=True)
sample_submission = sample_submission.merge(date_info, on='calendar_date', how='left')
sample_submission = sample_submission.merge(visitors, on=[
    'air_store_id', 'day_of_week', 'holiday_flg'], how='left')

missings = sample_submission.visitors.isnull()
sample_submission.loc[missings, 'visitors'] = sample_submission[missings].merge(
    visitors[visitors.holiday_flg==0], on=('air_store_id', 'day_of_week'), 
    how='left')['visitors_y'].values

missings = sample_submission.visitors.isnull()
sample_submission.loc[missings, 'visitors'] = sample_submission[missings].merge(
    visitors[['air_store_id', 'visitors']].groupby('air_store_id').mean().reset_index(), 
    on='air_store_id', how='left')['visitors_y'].values

sample_submission['visitors'] = sample_submission.visitors.map(pd.np.expm1)
sub2 = sample_submission[['id', 'visitors']].copy()
sub_merge = pd.merge(sub1, sub2, on='id', how='inner')

In [39]:
sub_merge['visitors'] = (sub_merge['visitors_x'] + sub_merge['visitors_y']* 1.1)/2
sub_merge[['id', 'visitors']].to_csv('submission_rs_recruit_v11_xgbm_v14_02.csv', index=False)
# LB 0.487